In [1]:
import numpy as np
import os
import random
import scipy
from scipy import signal
from numpy.fft import fft
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.initializers import he_normal
from tensorflow.python.keras.utils.vis_utils import plot_model

from sklearn.model_selection import train_test_split

import pandas as pd
import pickle

import seaborn as sns
sns.set(font='Yu Gothic')
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
def plot_confusion_matrix(test_y,pred_y,class_names,normalize=False, figsize=(16, 8)):
    cm = confusion_matrix(test_y,pred_y)
    # classes = class_names[unique_labels(test_y,pred_y)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    fig, ax = plt.subplots(figsize=figsize)
    im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    ax.figure.colorbar(im, ax=ax)
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           xticklabels=class_names,
           yticklabels=class_names,
           ylabel='True label\n',
           xlabel='\nPredicted label')
    fmt = '.2f' if normalize else 'd'
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j,
                    i,
                    format(cm[i, j], fmt),
                    ha="center",
                    va="center",
                    color="red", fontsize=16)
    fig.tight_layout()
    return ax

In [ ]:
train_Label = np.delete(np.load("../Data/センサ別npyファイル/train/train_Bag/train_Bag_LAbel.npy")[:, 0], 120845, 0).reshape([-1, 1])
Y_train = train_Label
train_Label.shape

In [ ]:
def load_npy(hold_position):
    mag = np.delete(np.load("../Output/train/train_" + hold_position + "/train_" + hold_position + "_glo_mag_norm_spectram.npy"), 120845, 0)[:, :40, :]
    mag = mag[(Y_train.reshape([-1]) == 1) | (Y_train.reshape([-1]) >= 5)]
    return mag

In [ ]:
train_Bag = load_npy("Bag")
train_Hips = load_npy("Hips")
train_Torso = load_npy("Torso")
train_Hand = load_npy("Hand")

train_Hand.shape

In [ ]:
X_train1 = np.concatenate([train_Bag, train_Hips, train_Torso, train_Hand], axis=0)
X_train1.shape

In [ ]:
X_train1 = X_train1.reshape([-1, X_train1.shape[1], X_train1.shape[2], 1])

X_train1.shape

In [ ]:
def load_npy(hold_position):
    lacc = np.delete(np.load("../Output/train/train_" + hold_position + "/train_" + hold_position + "_glo_lacc_norm_spectram.npy"), 120845, 0)[:, :64, :]
    lacc = lacc[(Y_train.reshape([-1]) == 1) | (Y_train.reshape([-1]) >= 5)]
    return lacc

In [ ]:
train_Bag = load_npy("Bag")
train_Hips = load_npy("Hips")
train_Torso = load_npy("Torso")
train_Hand = load_npy("Hand")

train_Hand.shape

In [ ]:
X_train2 = np.concatenate([train_Bag, train_Hips, train_Torso, train_Hand], axis=0)
X_train2.shape

In [ ]:
X_train2 = X_train2.reshape([-1, X_train2.shape[1], X_train2.shape[2], 1])

X_train2.shape

In [ ]:
def load_npy(hold_position):
    gyr = np.delete(np.load("../Output/train/train_" + hold_position + "/train_" + hold_position + "_glo_gyr_norm_spectram.npy"), 120845, 0)[:, :64, :]
    gyr = gyr[(Y_train.reshape([-1]) == 1) | (Y_train.reshape([-1]) >= 5)]
    return gyr

In [ ]:
train_Bag = load_npy("Bag")
train_Hips = load_npy("Hips")
train_Torso = load_npy("Torso")
train_Hand = load_npy("Hand")

train_Hand.shape

In [ ]:
X_train3 = np.concatenate([train_Bag, train_Hips, train_Torso, train_Hand], axis=0)
X_train3.shape

In [ ]:
X_train3 = X_train3.reshape([-1, X_train3.shape[1], X_train3.shape[2], 1])

X_train3.shape

In [ ]:
del train_Bag, train_Hips, train_Torso, train_Hand, train_Label, mag, gyr, lacc

In [ ]:
Y_train = np.concatenate([Y_train, Y_train, Y_train, Y_train], axis=0)
Y_train.shape

In [ ]:
Y_train = Y_train[(Y_train.reshape([-1]) == 1) | (Y_train.reshape([-1]) >= 5)]
Y_train.shape

In [ ]:
Y_train[Y_train.reshape([-1]) == 1] = 0

Y_train[Y_train.reshape([-1]) >= 5] = Y_train[Y_train.reshape([-1]) >= 5] - 4

np.unique(Y_train)

In [ ]:
x_train1, x_test1, y_train, y_test = train_test_split(X_train1, Y_train, test_size=0.3, shuffle=False)
x_train2, x_test2 = train_test_split(X_train2, test_size=0.3, shuffle=False)
x_train3, x_test3 = train_test_split(X_train3, test_size=0.3, shuffle=False)

x_train1.shape, x_test1.shape

In [ ]:
del X_train1, X_train2, X_train3

In [ ]:
def BuildModel(input_shape):
    input1 = layers.Input(shape=input_shape)
    x = layers.Conv2D(64, (3, 3), padding='valid', activation='relu', kernel_initializer=he_normal())(input1)
    x = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.BatchNormalization()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(16, activation='relu')(x)
    x = models.Model(inputs=input1, outputs=x)
    return x

In [ ]:
x1 = BuildModel(x_train1[0].shape)
x2 = BuildModel(x_train2[0].shape)
x3 = BuildModel(x_train3[0].shape)

combined = layers.concatenate([x1.output, x2.output, x3.output])

z = layers.Dense(64, activation='relu')(combined)
z = layers.Dense(16, activation='relu')(z)
z = layers.Dense(5, activation='softmax')(z)

model = models.Model(inputs=[x1.input, x2.input, x3.input], outputs=z)
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

In [ ]:
plot_model(model, show_shapes=True, show_layer_names=False)

In [ ]:
cp_cb = tf.keras.callbacks.ModelCheckpoint(filepath="../Output/CheckPoint/mag_norm/model_{epoch:02d}-{loss:.2f}-{accuracy:.2f}-{val_loss:.2f}-{val_accuracy:.2f}.hdf5", 
                                           monitor='val_loss', verbose=0, save_best_only=True, mode='auto')
es_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, verbose=0, mode='auto')
history = model.fit([x_train1, x_train2, x_train3], y_train, epochs=256, batch_size=512, validation_data=([x_test1, x_test2, x_test3], y_test), callbacks=[cp_cb, es_cb])

In [ ]:
# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
# plt.ylim((0, 3.5))
plt.show()

In [2]:
model = tf.keras.models.load_model("../Output/CheckPoint/mag_norm/model0527_14-0.81-0.69-1.54-0.46.hdf5")
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 40, 25, 1)]  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 64, 25, 1)]  0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 64, 25, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 38, 23, 64)   640         input_1[0][0]                    
____________________________________________________________________________________________

In [ ]:
# class_names = ['Still', 'Walking', 'Run', 'Bike', 'Car', 'Bus', 'Train', 'Subway']
class_names = ["Still", "Car", "Bus", "Train", "Subway"]
predict = model.predict_classes(X_test)
plot_confusion_matrix(Y_test, predict, class_names, False, (16, 8))
plt.grid(False)
f1_macro = f1_score(Y_test, predict, average='macro')
round(f1_macro, 3)

# testデータの読み込み→推定

In [3]:
test1 = np.load("../Output/test/test_glo_mag_norm_spectram.npy")[:, :40]
test1.shape

(57573, 40, 25)

In [4]:
test2 = np.load("../Output/test/test_glo_lacc_norm_spectram.npy")[:, :64]
test2.shape

(57573, 64, 25)

In [5]:
test3 = np.load("../Output/test/test_glo_gyr_norm_spectram.npy")[:, :64]
test3.shape

(57573, 64, 25)

In [6]:
test1 = test1.reshape([-1, 40, 25, 1])
test2 = test2.reshape([-1, 64, 25, 1])
test3 = test3.reshape([-1, 64, 25, 1])


In [7]:
predict = np.argmax(model.predict([test1, test2, test3]), axis=1)
predict

array([7, 1, 1, ..., 3, 3, 3], dtype=int64)

In [8]:
np.save("test_spectram_predict_0527", predict)